In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pythainlp.util import normalize
from pythainlp.ulmfit.utils import ThaiTokenizer
from pythainlp.corpus import stopwords
from pythainlp.tag import pos_tag

import pandas as pd
import numpy as np
import os, json, sys
import copy, glob
import pythainlp
from tqdm import tqdm
import re, string

# Text_to_tokens
- nomalize
- remove duplicate ending character
- remove url
- remove cr
- remove unwanted character
- remove stopwords
- to lowercase
- tokenize
- remove space 

In [2]:
def text_to_tokens(text, stop_words=[]):
    # nomalize
    text = pythainlp.util.normalize(text)
    
    # remove duplicate ending characters
    for m in re.finditer(r'([\u0E00-\u0E7F])(\1{2,})', text):
        text = text.replace(m.group(0),m.group(1),1)
        
    # remove url
    text = re.sub(r'http\S+', '', text)
    
    # remove CR
    text = text.replace('CR', '').replace('SR', '').replace('\xa0', '')
        
    # remove unwanted character
    pattern = re.compile(r"[^\u0E00-\u0E7Fa-zA-Z ]|ๆ")
    text = pattern.sub('',text)
    
    # lowercase
    text = text.lower()
    
    # tokenization
    tokens = pythainlp.tokenize.word_tokenize(text, engine='newmm', whitespaces=False)
    
    # remove stopwords
    tokens = [t for t in tokens if not t in stop_words] 
    
    return tokens

## pos_filter_noun
- filter noun from pos to find topics

In [3]:
def pos_filter_noun(tokens):
    postags =  pos_tag(tokens, engine = 'artagger')
    filterpos = []
    noun = ['NPRP','NCMN']
    for w in postags:
        
        # filter noun
        if w[1] in noun:
            filterpos.append(w[0])
        
    return filterpos


## fine_ques_type
- find type of question sentence


In [4]:
def fine_ques_type(text):
    
    # nomalize
    text = normalize(text)
    
    # question keywords

    where = ['ที่ไหน','อยู่ไหน','ตรงไหน','ไรดี'] 
    when = ['เมื่อไหร่','เปิดไหม','กี่โมง','กี่ทุ่ม','กี่นาที','กี่วัน','กี่เดือน','กี่ปี']
    why = ['ทำไม']
    who = ['ใคร']
    whose = ['ของใคร']
    which = ['อันไหน','หรือ']
    what = ['อะไร','แบบไหน','ทำไร','ไหน']
    how = ['อย่างไร','ยังไง','ทำไง','เท่าไหร่','เท่าไร','เป็นไง','มีกี่']
    rec = ['แนะนำ','รีวิว']
    yes_no = ['ใช่ไหม','ใช่มั้ย','ได้ไหม','หรือยัง','ได้ไหม','ไปไหม','ใช่ไหม','ได้มั้ย','มั้ย','หรือเปล่า','ไหม','ได้ป่าว','ได้ปะ','ได้รึเปล่า','มั๊ย','กันยัง']
    
    ques_types = []
    
    
            
    for w in where:
        if w in text:
            ques_types.append('where')
            break
            
    for w in when:
        if w in text:
            ques_types.append('when')
            break
            
    for w in why:
        if w in text:
            ques_types.append('why')
            break
            
    for w in who:
        if w in text:
            ques_types.append('who')
            break
            
    for w in whose:
        if w in text:
            ques_types.append('whose')
            break
            
    for w in which:
        if w in text:
            ques_types.append('which')
            break
            
    for w in what:
        if w in text and 'which' not in ques_types and 'where' not in ques_types:
            ques_types.append('what')
            break
            
    for w in how:
        if w in text:
            ques_types.append('how')
            break
            
    for w in rec:
        if w in text:
            ques_types.append('rec')
            break
            
    for w in yes_no:
        if w in text and 'when' not in ques_types:
            ques_types.append('yes_no')
            break
    
    return ques_types
        

In [5]:
df = pd.read_csv('./data/predicted-non-questions - question.csv',encoding='utf-8-sig')
df = df.drop(['label'],axis = 1)

df2 = pd.read_csv('./data/predited-questions - question and _0.75.csv',encoding='utf-8-sig')
result = df.append(df2,ignore_index=True)
result

,text,probability
0,มันแปลกหรอ ถ้าสมมติเราอยู่กับเพื่อนคนนี้แล้วเร...,0.5000
1,วิ่งหนีเหนื่อยมั้ย วิ่งตามเหนื่อยมาก,0.5001
2,เคยเจอพี่สิงที่มอมั้ยย (สิงโต),0.5001
3,ใครไปเจาะเลือดมั่งงงง,0.5001
4,วิ่งหนีเหนื่อยมั้ย\nวิ่งตามเหนื่อยมาก..,0.5001
5,ไหนบอกเราว่าเราเป็นความสุขของคุณไง\nแล้วตอนนี้...,0.5001
6,เจ้าแอพนี้มันเล่นยังไงนะ ;__;,0.5002
7,40000 คนตอนนี้เต็มยังค่ะ,0.5002
8,อีกนานไหม กว่าใจจะลืม,0.5002
9,เรามันก็แค่คนในไลน์\nจะไปสู้คนข้างกายเธอได้ยัง...,0.5002


In [6]:
def find_types(df,stopword):
    data = []
    for t in tqdm(df['text']):

        types = ','.join(fine_ques_type(t))
        
        tokens = text_to_tokens(t,stopword)
        topics = ','.join(pos_filter_noun(tokens))
        
        data.append({
            'text':t,
            'types':types,
            'keywords':topics
        })

    df2 = pd.DataFrame(data)
    return df2[['text','types','keywords']]


In [7]:
def find_stopwords(df):
    stopwords = []
    
    for index,row in df.iterrows():
        if row['label'] == 0:
            stopwords.append(row['Stopword'])
    return stopwords

## Experiment

In [8]:
# download stopword
# stopwords version 2
df = pd.read_csv('./data/stopwords.2.csv',encoding='utf-8-sig')
df.fillna(0,inplace = True)

stopwords = find_stopwords(df)
stopwords

['นี้',
 'นํา',
 'นั้น',
 'นัก',
 'ที่',
 'ทั้งนี้',
 'จาก',
 'จะ',
 'ความ',
 'ครั้ง',
 'คง',
 'น่า',
 'เข้า',
 'ถึง',
 'ต่อ',
 'ตั้งแต่',
 'ตั้ง',
 'ด้าน',
 'ด้วย',
 'อีก',
 'อาจ',
 'อย่าง',
 'แต่',
 'เมื่อ',
 'เพื่อ',
 'หลัง',
 'หนึ่ง',
 'ส่วน',
 'สุด',
 'ราย',
 'ขณะ',
 'ก็',
 'การ',
 'กัน',
 'กว่า',
 'จึง',
 'ไว้',
 'ไป',
 'ได้',
 'โดย',
 'แห่ง',
 'แล้ว',
 'และ',
 'ทั้ง',
 'เคย',
 'ขั้น',
 'ไง',
 'จง',
 'ซึ่งก็',
 'ซึ่งก็คือ',
 'ซึ่งได้แก่',
 'ณ',
 'ด้วย',
 'ด้วยเช่นกัน',
 'ด้วยที่',
 'ด้วยประการฉะนี้',
 'ด้วยเพราะ',
 'ด้วยว่า',
 'ด้วยเหตุที่',
 'ด้วยเหตุนั้น',
 'ด้วยเหตุนี้',
 'ด้วยเหตุเพราะ',
 'ด้วยเหตุว่า',
 'ด้วยเหมือนกัน',
 'ดั่ง',
 'ดังกับ',
 'ดั่งกับ',
 'ดังกับว่า',
 'ดั่งกับว่า',
 'ดังเก่า',
 'ดั่งเก่า',
 'ดังเคย',
 'ตลอดจน',
 'ตลอดถึง',
 'ตลอดทั้ง',
 'ตลอดทั่ว',
 'ตลอดทั่วถึง',
 'ตลอดทั่วทั้ง',
 'ตลอดไป',
 'ตลอดมา',
 'ถึงจะ',
 'ถึงเมื่อ',
 'ถึงแม้',
 'ถึงแม้จะ',
 'ถึงแม้ว่า',
 'ถึงอย่างไร',
 'เถอะ',
 'เถิด',
 'ทั้งนั้น',
 'ทั้งนั้นด้วย',
 'ทั้งนั้นเพราะ',
 'นอกจากที่',
 'นอ

In [9]:

result_df = find_types(result,stopwords)

100%|██████████| 19356/19356 [14:01<00:00, 23.01it/s]


In [10]:
result_df

,text,types,keywords
0,มันแปลกหรอ ถ้าสมมติเราอยู่กับเพื่อนคนนี้แล้วเร...,yes_no,"หรอ,เพื่อน,โลก,มีความสุข,ไหม,ทั้งโลก,ป่ะ,เหนื่อย"
1,วิ่งหนีเหนื่อยมั้ย วิ่งตามเหนื่อยมาก,yes_no,"วิ่งหนี,เหนื่อย,มั้ย,เหนื่อย"
2,เคยเจอพี่สิงที่มอมั้ยย (สิงโต),yes_no,"พี่,สิง,มอ,มั้ย,ย,สิงโต"
3,ใครไปเจาะเลือดมั่งงงง,who,"เลือด,มั่ง"
4,วิ่งหนีเหนื่อยมั้ย\nวิ่งตามเหนื่อยมาก..,yes_no,"วิ่งหนี,เหนื่อย,มั้ย,เหนื่อย"
5,ไหนบอกเราว่าเราเป็นความสุขของคุณไง\nแล้วตอนนี้...,what,"ความสุข,ตอนนี้,ล่ะ"
6,เจ้าแอพนี้มันเล่นยังไงนะ ;__;,how,"เจ้า,แอ,พ,ยังไง"
7,40000 คนตอนนี้เต็มยังค่ะ,,ตอนนี้
8,อีกนานไหม กว่าใจจะลืม,yes_no,"ไหม,ใจ,ลืม"
9,เรามันก็แค่คนในไลน์\nจะไปสู้คนข้างกายเธอได้ยัง...,how,"ไลน์,ข้าง,กาย"
